In [2]:
import tools as tl
from selenium.webdriver.support.select import Select
import requests
import pandas as pd
import configparser
import datetime

In [2]:
def get_config_data(**kwargs):
    """
    This function reads a configuration file and returns the value of a specified field.
    
    :param path: The path to the configuration file that needs to be read
    :return: the data from the specified field and value in the configuration file located at the
    specified path.
    """
    #Abrindo o arquivo de configurações
    arq = configparser.RawConfigParser()
    arq.read(r'C:\Users\vitor\OneDrive\Documentos\Python Scripts\abreu_network_robot\config.txt')

    #Pegando os dados do arquivo
    data = arq.get(kwargs['field'], kwargs['value'])

    return data


In [3]:
# Função para substituir os emails pelo nome do cliente
def replace_emails_with_names(email, customer_dict):
    
    for name, emails in customer_dict.items():
        if (email.lower() in [em.lower() for em in emails]):
            return name
    return email

In [4]:
def create_customer_dict(customers_list, df_all):
    customer_dict = {}

    for customer in customers_list: 
        df_filt = df_all.loc[df_all['To'].str.contains(customer)]
        customer_dict[customer] = df_filt['To'].value_counts().index.to_list()
        #df_filt.to_excel(f'{OUTPUT_PATH}\{customer}.xlsx', index=False)
    
    return customer_dict

In [5]:
def replace_custom_names(customer_names):
    diff_custom = {
        'aguia':'aguiasecuritizadora',
        'bombinhassummer':'bombinhassummerbeach', 
        'dutoclean': 'dutocleanbauru',
        'fabricenter': 'fabricenter470',
        'fazcomunicacao': 'fazcomunicacaovisual',
        'galli': 'galliatacadista',
        'impacto': 'impactoesports',
        'jcleletronica': 'jcleletronicaindustrial',
        'mailinspector': 'filter.mailinspector',
        'plantfort': 'plantfortfertil',
        'primedigital': 'primemidiadigital',
        'quinta': 'quintadonino',
        'raysolar': 'raysolarbrasil',
        'reset': 'resetmairinque',
        'revar': 'revarcondicionado',
        'rodoflex': 'rodoflexpneus',
        'sany': 'sanydobrasil',
        'supernagai': 'supermercadosnagai',
        'agrotecagro': 'agrotecagroquimica',
        'barulhinhobomchips': 'barulhinhobom',
        'marcospaulo': 'marcospauloimoveis',
        'cortedobra': 'cortedobraseguranca',
        'ortobaby': 'ortobabycalcados',
        'arinteli': 'arinteligenciatributaria',
    }   

    for diff_name in diff_custom.keys():
        list_index = customer_names.index(diff_name)
        customer_names[list_index] = diff_custom[diff_name]
    
    return customer_names

In [6]:
diff_custom = {
    'aguia':'aguiasecuritizadora',
    'bombinhassummer':'bombinhassummerbeach', 
    'dutoclean': 'dutocleanbauru',
    'fabricenter': 'fabricenter470',
    'fazcomunicacao': 'fazcomunicacaovisual',
    'galli': 'galliatacadista',
    'impacto': 'impactoesports',
    'jcleletronica': 'jcleletronicaindustrial',
    'mailinspector': 'filter.mailinspector',
    'plantfort': 'plantfortfertil',
    'primedigital': 'primemidiadigital',
    'quinta': 'quintadonino',
    'raysolar': 'raysolarbrasil',
    'reset': 'resetmairinque',
    'revar': 'revarcondicionado',
    'rodoflex': 'rodoflexpneus',
    'sany': 'sanydobrasil',
    'supernagai': 'supermercadosnagai',
    'agrotecagro': 'agrotecagroquimica',
    'barulhinhobomchips': 'barulhinhobom',
    'marcospaulo': 'marcospauloimoveis',
    'cortedobra': 'cortedobraseguranca',
    'ortobaby': 'ortobabycalcados',
    'arinteli': 'arinteligenciatributaria',
    
}


In [7]:
def get_config_data(path, **kwargs):
    
    #Abrindo o arquivo de configurações
    arq = configparser.RawConfigParser()
    arq.read(path)
    
    #Pegando os dados do arquivo
    data = arq.get(kwargs['field'], kwargs['value'])
    
    return data

In [8]:
def replace_numeric_chars(string):
    replacements = {'0': 'o', '1': 'i', '3': 'e', '4': 'a', '5': 's', '7': 't', '8': 'b'}
    for char, replacement in replacements.items():
        if string not in ['5stintas','c3']:
            string = string.replace(char, replacement)
    return string

In [9]:
def format_date_api():
    """
    This Python function formats the current date and the next day's date in a specific
    format for use
    in an API.
    :return: a tuple with two strings: the current date formatted as "mm/dd/yyyy" and the next day's
    date formatted as "mm/dd/yyyy", both with the forward slash replaced by "-".
    """

    # Obtém a data atual
    data_atual = datetime.date.today()
    data_atual_formatada = data_atual.strftime("%m-%d-%Y")
    data_atual_formatada = data_atual_formatada.replace('-', '%2F')

    # Adiciona um dia
    data_nova = data_atual + datetime.timedelta(days=1)
    data_nova_formatada = data_nova.strftime("%m-%d-%Y")
    data_nova_formatada = data_nova_formatada.replace('-', '%2F')
    return data_atual_formatada, data_nova_formatada


In [10]:
URL = 'https://filter.mailinspector.com.br/login/index.php'
API_URL = 'https://filter.mailinspector.com.br/login/mailLogViewer.php'
OUTPUT_PATH = r'C:\Users\vitor\OneDrive\Documentos\Python Scripts\abreu_network_robot\files'
user ='admin'
senha = 'N6wq@Ks72'

In [11]:
tl.clean_dir(OUTPUT_PATH)

Limpando arquivos gerais
log_view.csv removido com sucesso.
log_view.txt removido com sucesso.


### 1 Etapa

- Acessar o portal
- Salvar a lista de clientes 
- Salvar os cookies para passar na API

In [12]:
driver = tl.create_driver(headless=False)
driver.get(URL)

In [13]:
tl.clicking(element='clicando em login', path='email', by='name', driver=driver).click()
tl.clicking(element='passando o user', path='email', by='name', driver=driver).send_keys(user)

tl.clicking(element='clicando em senha', path='password', by='name', driver=driver).click()
tl.clicking(element='passando a senha', path='password', by='name', driver=driver).send_keys(senha)

tl.clicking(element='clicando no botao login', path="//input[@type='submit']", driver=driver).click()

>>>Sucesso clicando em login>>>
>>>Sucesso passando o user>>>
>>>Sucesso clicando em senha>>>
>>>Sucesso passando a senha>>>
>>>Sucesso clicando no botao login>>>


In [14]:
tl.clicking(element='Selecionando Linked Accounts', path="//*[text()='Linked Accounts']", driver=driver).click() 

>>>Sucesso Selecionando Linked Accounts>>>


In [15]:
drop_down = tl.clicking(element='selecionando o selector com todos os clientes', path='hijackName', by='name', driver=driver)
drop_down_obj = Select(drop_down)

>>>Sucesso selecionando o selector com todos os clientes>>>


In [16]:
customers_list = [opt.text for opt in drop_down_obj.options]
customers_list.pop(0)
customers_list = [em.lower() for em in customers_list]
customers_list = list(map(lambda char: replace_numeric_chars(char), customers_list))
customers_list = replace_custom_names(customers_list)

In [17]:
cookies = tl.get_cookies(driver)

In [18]:
driver.quit()

### 2 Etapa
- Configurar os paramentros da API
- Acessar os reports dos clientes via API
- salvar como arquivo csv para facilitar a modelagem

In [19]:
headersList = {
 "Cookie": f"PHPSESSID={cookies['PHPSESSID']}; lang=pt_BR",
 "Referer": "https://filter.mailinspector.com.br/login/mailLogViewer.php", 
}

#formatando as datas para passar na API
dia_atual, proximo_dia = format_date_api()

#Requisição na api para baixar os dados dos clientes
response = requests.get(API_URL+f'?from={dia_atual}&to={proximo_dia}&page=-1',
                        headers=headersList,
                        timeout=100)

result = response.text

In [20]:
with open(f'{OUTPUT_PATH}\log_view.txt', 'w') as f:
    f.write(result)

In [21]:
df = pd.read_csv(f'{OUTPUT_PATH}\log_view.txt')
df.to_csv(f'{OUTPUT_PATH}\original.csv', index=False, sep=',')
df.head()

,Date,From,To,Subject,Action,Score,Source IP
0,2023-06-28 22:52:52,qarecebnfefiat@nfe.fgbr,nfe@breitkopf.com.br,NaN,Rejected,NaN,187.0.250.144
1,2023-06-28 22:52:52,qarecebnfefiat@nfe.fgbr,nfe@breitkopf.com.br,NaN,Rejected,NaN,187.0.250.144
2,2023-06-28 22:52:52,bounces+6102634-2596-societario02=ccop.com.br@...,societario02@ccop.com.br,NaN,Rejected,NaN,167.89.107.163
3,2023-06-28 22:52:50,contadigital6366@vivo.com.br,locacoes@fladafi.com.br,NaN,Blacklisted,NaN,191.240.2.103
4,2023-06-28 22:52:47,qarecebnfefiat@nfe.fgbr,nfe@breitkopf.com.br,NaN,Rejected,NaN,187.0.250.144


### 4 Etapa
- Organizar o arquivo extraido da API
- Alterar a coluna 'To' que são os emails, para os nomes dos clientes

In [22]:
customer_dict = {}
for customer in customers_list:
    customer_dict[customer] = [email for email in df['To'] if (email.endswith('@'+customer.lower()+'.com.br')) or (email.endswith('@'+customer.lower()+'.com'))]

In [23]:
#customer_dict = create_customer_dict(customers_list, df)
df_customer = pd.DataFrame({"Cliente":list(customer_dict.keys()),
                            "Emails":list(customer_dict.values())})

df_customer.to_excel(f'{OUTPUT_PATH}\customers_emails.xlsx', index=False)

In [24]:
df_customer

,Cliente,Emails
0,5stintas,"[sama@5stintas.com.br, financeiro@5stintas.com..."
1,abreuhost,"[gilson@abreuhost.com.br, suporte@abreuhost.co..."
2,abreunetworks,"[registro.br@abreunetworks.com.br, financeiro@..."
3,abreuweb,[]
4,adhering,"[futsal@adhering.com.br, marketing@adhering.co..."
...,...,...
297,votocont,[]
298,westfol,"[contato@westfol.com.br, contato@westfol.com.b..."
299,zapplin,[]
300,zigues,[]


In [25]:
df_all = pd.DataFrame({'Date': df['Date'], 
                       'From':df['From'], 
                       'To': df['To'].apply(replace_emails_with_names, customer_dict=customer_dict), 
                       'Action': df['Action']})
df_all.head()

,Date,From,To,Action
0,2023-06-28 22:52:52,qarecebnfefiat@nfe.fgbr,breitkopf,Rejected
1,2023-06-28 22:52:52,qarecebnfefiat@nfe.fgbr,breitkopf,Rejected
2,2023-06-28 22:52:52,bounces+6102634-2596-societario02=ccop.com.br@...,ccop,Rejected
3,2023-06-28 22:52:50,contadigital6366@vivo.com.br,fladafi,Blacklisted
4,2023-06-28 22:52:47,qarecebnfefiat@nfe.fgbr,breitkopf,Rejected


In [26]:
df_all.to_csv(f'{OUTPUT_PATH}\log_view.csv', sep=',', index=False)

### 5 Etapa
- Extrair do arquivo organizado os dados a serem mostrado na dashboard

In [40]:
#quantidade de emaisl por acao realizada
df_filt = df_all.loc[df_all['To']=='dannil']
teste = df_filt['Action'].value_counts()
teste

Action
Sent            326
Rejected         73
Blacklisted      52
Blocked SPAM      3
Bounced           1
Name: count, dtype: int64

In [36]:
#total de emails recebidos
df_filt = df_all.loc[df_all['To']=='dannil']


210

In [42]:
#contas que mais madaram email x ação realizada
df_filt = df_all.loc[df_all['To']=='adhering']
serie = df_filt['From'].value_counts()
email_series = serie.loc[serie>1].index.to_list()
df_from = df_filt.loc[df_filt['From'].isin(email_series)]
qtd_series = df_from['From'].value_counts()
df_tres = pd.DataFrame({'From': qtd_series.index, 'Qtd': qtd_series.values, 'Actions': df_from.drop_duplicates(subset=['From'])['Action']})
df_tres

,From,Qtd,Actions
5373,check@mx-check.com,21,Sent
8431,q9xlmfodt5k6zy@nowacia.ru,21,Blacklisted
9072,8adm64k4hdbr8zb@milkindustry.ru,21,Sent
9310,nfse@emailsimpliss.com.br,8,Sent
10667,bounces+9647731-b2b2-almoxarifado=adhering.com...,6,Sent
11221,wtfeventos@hotmail.com,6,Rejected
12314,rh@jordancontabilidade.com.br,4,Sent
13674,nilson.haas@grafica43.com.br,4,Sent
14914,pm_bounces@pm.mtasv.net,4,Blacklisted
15433,baumtec@baumtec.com.br,4,Rejected


### ZONA DE TESTES E AJUSTES

In [29]:
df_errados = df_all.loc[df_all['To'].str.contains('@')]
df_errados['To'].value_counts()

To
central@casadecarnesaraujo.com.br      371
mileshopeeg@lardasfitas.com.br          98
shopee@lardasfitas.com.br               86
advogada@viviane.adv.br                 83
contato@lardasfitas.com.br              71
                                      ... 
zd7k9b6q1upqpre@c3radiologia.com.br      1
9kindbi16vzow@c3radiologia.com.br        1
emfbwfdtv4db@c3radiologia.com.br         1
q410tr6yw4pvy5s@c3radiologia.com.br      1
postmaster@patiocorina.com.br            1
Name: count, Length: 666, dtype: int64

In [30]:
error_list = []

for cust in df_errados['To'].drop_duplicates():
    split_str = cust.split('@')[1]
    if "@"+split_str not in error_list:
        error_list.append("@"+split_str)

In [31]:
error_list

['@lardasfitas.com.br',
 '@gressusseguros.com.br',
 '@rentalenergybrasil.com.br',
 '@nfendd.com.br',
 '@basesc.eng.br',
 '@simpleslocadora.com.br',
 '@outlook.de',
 '@renatodiasplanner.com.br',
 '@dell.com',
 '@otto.ind.br',
 '@reseteletronicaindustrial.com.br',
 '@viviane.adv.br',
 '@duffles.net.br',
 '@gmail.com',
 '@unixsaude.com.br',
 '@dimasdona.com.br',
 '@outlookweb.com.br',
 '@greaselubrificantes.com.br,vendas',
 '@greaselubrificantes.com.br',
 '@pirosolucoes.com.br',
 '@VIVIANE.ADV.BR',
 '@renovaagroibitinga.com.br',
 '@spaceengenharia.eng.br',
 '@cnl.inf.br',
 '@abreubusiness.com.br',
 '@casadecarnesaraujo.com.br',
 '@ltbgrupodobrasil.com.br',
 '@terra.com.br',
 '@valius.com.br',
 '@elder-th.eti.br',
 '@fuabc.org.br',
 '@RENATODIASPLANNER.COM.BR',
 '@SMSR.COM.BR',
 '@celesc.com.br',
 '@localiza.com',
 '@paulorogerioimoveis.com.br',
 '@mdsolucoes.net.br',
 '@disman.com.br',
 '@grupovamos.com.br',
 '@greaseespecial.com.br',
 '@econeteditora.com.br',
 '@OTTO.IND.BR,DAIANE',
 '@O

### Adicionando os IP's na Blacklist

In [1]:
import os
import datetime
import requests
import pandas as pd
import tools as tl

#Variaveis fixas, que serão utilizadas em todo o processo
URL = 'https://filter.mailinspector.com.br/login/index.php'
API_URL = 'https://filter.mailinspector.com.br/login/mailLogViewer.php'
OUTPUT_PATH = r'C:\Users\no\Python Programs\abreu_network_robot\files'
CONFIG_PATH = r'C:\Users\no\Python Programs\abreu_network_robot\config.txt'

In [2]:
def get_portal_cookies():
    """
    Esta função faz login em um portal da web, 
    recupera uma lista de clientes e retorna os cookies da sessão.
    :retorno: uma tupla contendo uma lista de clientes e um dicionário de cookies.
    """
    #Acessando os dados para logar no portal
    user = tl.get_config_data('LOGIN', 'user', CONFIG_PATH)
    psw = tl.get_config_data('LOGIN', 'password', CONFIG_PATH)

    #Cria o objeto driver, responsavel por acessar os dados dentro da web
    driver = tl.create_driver_chrome(headless=False)
    driver.get(URL)

    #Logando dentro do portal
    tl.clicking(element='clicando em login',path='email',btype='name',driver=driver).click()
    tl.clicking(element='passando o user',path='email',btype='name',driver=driver).send_keys(user)
    tl.clicking(element='clicando em senha',path='password',btype='name',driver=driver).click()
    tl.clicking(element='passandosenha',path='password',btype='name',driver=driver).send_keys(psw)
    tl.clicking(element='clicando no login',path="//input[@type='submit']",driver=driver).click()

    #Armazendo os cookies do portal
    cookies = tl.get_cookies(driver)

    return cookies, driver

def format_date_api():
    """
    Esta função formata a data atual e a data do próximo dia em um formato específico.
    :retorno: uma tupla com duas strings: a data atual formatada como "mm/dd/yyyy" 
    e a data do próximo dia formatada como "mm/dd/yyyy",
    ambas com a barra inclinada substituída por "-".
    """

    # Obtém a data atual
    data_atual = datetime.date.today()
    data_atual_formatada = data_atual.strftime("%m-%d-%Y")
    data_atual_formatada = data_atual_formatada.replace('-', '%2F')

    # Adiciona um dia - proximo dia
    data_nova = data_atual + datetime.timedelta(days=1)
    data_nova_formatada = data_nova.strftime("%m-%d-%Y")
    data_nova_formatada = data_nova_formatada.replace('-', '%2F')

    #Retorna o dia atual e o proximo dia foramatado
    return data_atual_formatada, data_nova_formatada

def extract_emails_log(authorization):
    """
    The function extrai o log de emails enviados filtrando pela data
    """

    #formatando as datas para passar na API
    dia_atual, proximo_dia = format_date_api()

    #Criando o dicionario para passar como paramentro
    header_list = {
    "Cookie": f"PHPSESSID={authorization['PHPSESSID']}; lang=pt_BR",
    "Referer": API_URL, 
    }

    #Requisição na api para baixar os dados dos clientes
    response = requests.get(API_URL+f'?from={dia_atual}&to={proximo_dia}&page=-1',
                            headers=header_list,
                            timeout=100)
    result = response.text

    #Salvando o resultado da API no arquivo txt, ja no padrao csv
    with open(f'{OUTPUT_PATH}/emails_log.txt', 'w', encoding='utf-8') as file:
        file.write(result)


In [3]:
#Pegando os cookies do portal
authorization, driver =  get_portal_cookies()
extract_emails_log(authorization)

#Lendo o arquivo txt como csv
df_api = pd.read_csv(f'{OUTPUT_PATH}/log_view.txt')
df_api.to_excel(f'{OUTPUT_PATH}/teste.xlsx', index=False)

#Filtra o arquivo extraido mantendo apenas valores "Blacklisted"
df_filt = df_api.loc[df_api['Action']=='Blacklisted']
df_filt

>>>Sucesso clicando em login>>>
>>>Sucesso passando o user>>>
>>>Sucesso clicando em senha>>>
>>>Sucesso passandosenha>>>
>>>Sucesso clicando no login>>>


,Date,From,To,Subject,Action,Score,Source IP
120,2023-08-17 11:51:32,www-data@vngdlkwv7.poijcymkt.online,sylvio@birimesas.com.br,NaN,Blacklisted,NaN,164.92.183.59
153,2023-08-17 11:51:19,mercadopago43518@xcvbsdgt1.mpvalidesxc.com,nfe_vwcoitj@breitkopf.com.br,NaN,Blacklisted,NaN,138.197.114.4
156,2023-08-17 11:51:12,www-data@vngdlkwv5.poijcymkt.online,transpompiani@transpompiani.com.br,NaN,Blacklisted,NaN,170.64.166.188
160,2023-08-17 11:51:12,www-data@vngdlkwv9.poijcymkt.online,silmara@soldatopo.com.br,NaN,Blacklisted,NaN,164.92.255.201
161,2023-08-17 11:51:11,bisoy@gmail.com,garantiabru@breitkopf.com.br,NaN,Blacklisted,NaN,190.181.23.186
...,...,...,...,...,...,...,...
49297,2023-08-17 00:00:43,0100018a01702801-fb01cdec-cb72-48ac-9135-c6837...,wladimir.goncalves@gsoconsulting.com.br,NaN,Blacklisted,NaN,54.240.65.40
49331,2023-08-17 00:00:23,cjgfo@iwvm7.atendimentosx.net,info@birimesas.com.br,NaN,Blacklisted,NaN,139.59.112.159
49336,2023-08-17 00:00:19,mndsd@iwvm22.atendimentosx.net,info@casm.com.br,NaN,Blacklisted,NaN,170.64.164.79
49391,2023-08-17 00:00:06,recepcao25@recepcao25.parapointerbrasil.com,djalma@plantfortfertil.com.br,NaN,Blacklisted,NaN,165.232.164.187


In [4]:
tl.clicking(element="Selecionando Blacklist", path='//*[@id="seg-menuce"]/ul/li[12]', driver=driver).click()

>>>Sucesso Selecionando Blacklist>>>


In [5]:
tl.clicking(element='Passado a lista de IPs, dentro do portal',path='entries',btype='name', driver=driver).send_keys("\n".join(df_filt['Source IP'].drop_duplicates()))

>>>Sucesso Passado a lista de IPs, dentro do portal>>>


In [6]:
tl.clicking(element='Clicando no botãoo de adcicionar', path='add_blacklist', btype='name', driver=driver).click()

>>>Sucesso Clicando no botãoo de adcicionar>>>


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=116.0.5845.111)
Stacktrace:
	GetHandleVerifier [0x00007FF6E4A652A2+57122]
	(No symbol) [0x00007FF6E49DEA92]
	(No symbol) [0x00007FF6E48AE3AB]
	(No symbol) [0x00007FF6E489C01A]
	(No symbol) [0x00007FF6E489BF63]
	(No symbol) [0x00007FF6E489A9E5]
	(No symbol) [0x00007FF6E489B31E]
	(No symbol) [0x00007FF6E48A80CB]
	(No symbol) [0x00007FF6E48B7F31]
	(No symbol) [0x00007FF6E48BC0DA]
	(No symbol) [0x00007FF6E489B8E7]
	(No symbol) [0x00007FF6E48B7E0A]
	(No symbol) [0x00007FF6E491EBE9]
	(No symbol) [0x00007FF6E4906DB3]
	(No symbol) [0x00007FF6E48DD2B1]
	(No symbol) [0x00007FF6E48DE494]
	GetHandleVerifier [0x00007FF6E4D0EF82+2849794]
	GetHandleVerifier [0x00007FF6E4D61D24+3189156]
	GetHandleVerifier [0x00007FF6E4D5ACAF+3160367]
	GetHandleVerifier [0x00007FF6E4AF6D06+653702]
	(No symbol) [0x00007FF6E49EA208]
	(No symbol) [0x00007FF6E49E62C4]
	(No symbol) [0x00007FF6E49E63F6]
	(No symbol)

In [7]:
len(df_filt['Source IP'].drop_duplicates())

1338

In [8]:
driver.quit()